# TO DO 

- Time and see how long it takes to extend the cache
    - How long to get to 710k, 750k, 800k, 1M
- Set a cutoff
    - If rdp_constants.max() > cutoff:
        - don't extend cache
        - directly calculate epsilon for each value in the array
    - else:
        - no changes needed.

In [1]:
from syft.core.adp.data_subject_ledger import DataSubjectLedger as DSL
from syft.core.adp.data_subject_ledger import convert_constants_to_indices

In [2]:
import jax
from jax import numpy as jnp
from jax import vmap

In [3]:
dsl = DSL()

In [4]:
import numpy as np

In [5]:
random_rdps = np.random.randint(low=10, high=2000000, size=(5,5))
random_rdps

array([[ 126170, 1372888, 1680138, 1621071, 1669914],
       [  23814,   45410, 1271573, 1493824,  826368],
       [1775059,  405897,  896659,  638858,  552236],
       [1230473,  227965, 1482800, 1735377,  150249],
       [1791002, 1374322, 1710398,  457540,  675690]])

In [6]:
dsl._get_epsilon_spend(random_rdps)

DeviceArray([[ 128804.96729901, 1381591.50305274, 1689766.90350945,
              1630529.0283855 , 1679513.54398854],
             [  24956.4364182 ,   46989.06632575, 1279948.98697805,
              1502903.00308071,  833119.21865791],
             [1784956.32726948,  410626.9586474 ,  903691.71950911,
               644793.39399859,  557753.98122101],
             [1238712.41702958,  231508.47998058, 1491845.41937527,
              1745163.00696351,  153124.85214252],
             [1800943.70157037, 1383030.05033524, 1720113.27943796,
               462562.17218861,  681794.2460214 ]], dtype=float64)

In [6]:
dsl._fetch_eps_spend_for_big_rdp(jnp.array([1, 10, 1000000])) 

DeviceArray([5.37271206e-02, 3.00498916e+01, 1.00742725e+06], dtype=float64)

In [7]:
dsl._fetch_eps_spend_for_big_rdp(random_rdps)

DeviceArray([[ 128014.77467909, 1069141.35835386, 1409639.741744  ,
               308691.74411929, 1554534.20507629],
             [  13091.10246689, 1712981.04765586,  362334.15056972,
              1149198.91012423, 1502788.65642435],
             [1624126.37499951, 1799872.73518312, 1649107.94958243,
              1678541.75672484, 1227612.26803043],
             [ 199272.20971234, 1494575.72329699,  185703.27787834,
              1029227.53474668, 1893891.87536269],
             [ 404933.8285889 ,  575999.03737368,   30476.69569434,
              1253691.28416044, 1906400.6009029 ]], dtype=float64)

In [4]:
vectorized_result = vmap(dsl._get_optimal_alpha_for_constant)

In [5]:
vectorized_result(jnp.ones(10))

TracerArrayConversionError: The numpy.ndarray conversion method __array__() was called on the JAX Tracer object Traced<ShapedArray(float64[])>with<BatchTrace(level=1/0)> with
  val = DeviceArray([14.4292162, 14.4292162, 14.4292162, 14.4292162, 14.4292162,
             14.4292162, 14.4292162, 14.4292162, 14.4292162, 14.4292162],            dtype=float64)
  batch_dim = 0
This Tracer was created on line /home/e/PycharmProjects/PySyft/packages/syft/src/syft/core/adp/data_subject_ledger.py:350 (fun)
See https://jax.readthedocs.io/en/latest/errors.html#jax.errors.TracerArrayConversionError

In [3]:
import numpy as np

In [6]:
!pwd

/home/e/PycharmProjects/PySyft/notebooks/Experimental/Ishan


In [10]:
cache = np.load("../../../packages/syft/src/syft/cache/constant2epsilon_1200k.npy")

In [11]:
cache[0]

0.05372712063485988

In [12]:
b = np.random.choice([True, False], (5,5))
b

array([[False, False, False, False, False],
       [False, False, False, False, False],
       [ True,  True, False,  True, False],
       [False, False,  True,  True, False],
       [ True,  True, False, False,  True]])

In [16]:
b ^ 1

array([[1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1],
       [0, 0, 1, 0, 1],
       [1, 1, 0, 0, 1],
       [0, 0, 1, 1, 0]])

In [9]:
np.maximum(convert_constants_to_indices(np.array([0])), 0)

array([0])

In [7]:
cache_index = convert_constants_to_indices(rdp_constant_array=np.array([0.0001]))
cache[cache_index]

array([696220.83590776])

In [36]:
np.float64

numpy.float64

In [ ]:
np.flo

In [39]:
np.finfo(np.float64).max

1.7976931348623157e+308

In [42]:
np.finfo(np.float32).max

3.4028235e+38

In [30]:
cache.dtype

dtype('float64')

In [14]:
cache_index = convert_constants_to_indices(rdp_constant_array=np.array([49, 50]))

In [15]:
cache[cache_index[0]]

97.64431643112496

In [16]:
cache[cache_index[1]]

99.16982082691798

In [4]:
from time import time

In [27]:
def simulate():
    output_times = []
    for new_values in [10_000, 50_000, 100_000]:
        t0 = time()
        cache_size = len(dsl._cache_constant2epsilon)
        dsl._increase_max_cache(cache_size + new_values)
        tf = time()
        output_times.append(tf-t0)
        print(f"Time taken for {new_values} = {tf - t0}s")
    return output_times

In [28]:
times = simulate()

Time taken for 10000 = 4.212839603424072s
Time taken for 50000 = 18.544428825378418s
Time taken for 100000 = 33.84507870674133s


In [5]:
len(dsl._cache_constant2epsilon)

1200000

In [6]:
t0 = time()
dsl._increase_max_cache(1350000)
tf = time()
print(tf-t0)

53.744325399398804


In [26]:
len(dsl._cache_constant2epsilon)

1210000